In [1]:
print("Ok")




Ok


In [ ]:
print("Ok")

Ok


In [1]:
%pwd

'c:\\My Area\\Projects\\GenAI_Learning\\Medical Chatbot\\Medical_Chatbot-with-LLMs-LangChain-Pinecone-Flask-Aws\\research'

In [5]:
import os
os.chdir("../")

In [6]:
%pwd


'c:\\My Area\\Projects\\GenAI_Learning\\Medical Chatbot\\Medical_Chatbot-with-LLMs-LangChain-Pinecone-Flask-Aws'

In [7]:
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# Extract text from PDF Files
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf_files("data")

In [11]:
# extracted_data

In [10]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    Containing only 'source' in metadata and the orginal page_content."""
    minimal_docs: list[Document] = []
    for doc in docs:
        src= doc.metadata.get("source")
        minimal_docs.append(
            Document(page_content=doc.page_content, metadata={"source": src})
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [40]:
# minimal_docs



In [13]:
# split the documents into smaller chunks

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks

In [14]:
text_chunks = text_split(minimal_docs)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [15]:
# from langchain.embeddings import HuggingFaceEmbeddings   OLD
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download & return HuggingFace Embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings
embedding = download_embeddings()

C:\Users\abhis\AppData\Local\Temp\ipykernel_10700\1908732076.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [16]:
vector = embedding.embed_query("Hello world")
# vector



In [17]:
print("Ok")

Ok


In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [18]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [19]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [20]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [25]:
pc

In [21]:
from pinecone import ServerlessSpec
index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
# Load Exisiting Index
from langchain_pinecone import PineconeVectorStore
# embed each chunk and insert  the embeddings into pinecone index
docsearch = PineconeVectorStore.from_existing_index(
     index_name=index_name,
     embedding=embedding
)

### Add More Data to  the Existing PineCone Index

In [29]:
dswith = Document(
    page_content="DswithAnkesh is a youtube channel that provides tutorials on programming, web development, and data science.",
    metadata={"source": "Youtube"}
)

In [30]:
docsearch.add_documents([dswith])

['d50de730-bae5-46e3-88f9-e46beb2679fd']

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [42]:
retrieved_docs = retriever.invoke("What is Acne")
retrieved_docs

[Document(id='c896f570-a01b-4743-a1a3-62efd16bf5e9', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2b70a9ee-6ef0-441c-919e-e2e1132281d5', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='bf6270b2-8d0e-45a4-a929-2d185a85e8ed', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any')]

In [80]:
from dotenv import load_dotenv
import os
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemini model
LLM = ChatGoogleGenerativeAI(
    google_api_key=GOOGLE_API_KEY, 
    model="gemini-2.5-flash", 
    temperature=0
)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are medcial assistant for  question_answering task"
"Use the following pieces of retrevieved context to answer"
"the question. If you don't know the answer, just say that you don't know, don't try to make up an answer."
"Use three sentences maximum to answer the question & keep the answer concise"
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
    )

In [39]:
question_answering_chain = create_stuff_documents_chain(LLM, prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [45]:
response = rag_chain.invoke({"input": "What is acromegaly & gigantisim?"})
print(response["answer"])

Acromegaly is a disorder characterized by the abnormal release of a specific chemical from the pituitary gland in the brain. This leads to increased growth in bone and soft tissue, along with other bodily disturbances. The provided text defines acromegaly but does not offer a separate definition for gigantism.
